In [1]:
!python -m pip install numpy pandas prophet matplotlib

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from prophet import Prophet
from datetime import timedelta

mpl.style.use("seaborn")

ModuleNotFoundError: No module named 'numpy'

In [ ]:
dataframe = pd.read_csv(
    "data/germany_vaccinations_timeseries_v2.tsv",
    sep="\t",
    parse_dates=["date"]
)
dataframe.head()

In [ ]:
df_german_holidays = pd.read_csv(
    "data/german_holidays.csv",
    parse_dates=["Tag"],
    sep=";"
    )["Tag"]

In [ ]:
df_kumulative = dataframe[["date", "dosen_kumulativ"]]

column_rename_schema = {
    'date':'ds',
    'dosen_kumulativ':'y',
}

df_kumulative.rename(columns = column_rename_schema, inplace = True)

df_kumulative.info()

In [ ]:
df_kumulative.plot(kind="line", x="ds", y="y", figsize=(15,10))

In [ ]:
holidays = pd.DataFrame({
  'holiday': 'Deutscher Feiertag',
  'ds': df_german_holidays,
  'lower_window': 0,
  'upper_window': 1,
})

prophet = Prophet(weekly_seasonality=True, holidays=holidays)

prophet.add_country_holidays(country_name = 'DE')

prophet.fit(df_kumulative)

future = prophet.make_future_dataframe(periods=200)

forecast = prophet.predict(future)

fig_forecast = plt.figure(dpi=200)

fig_forecast.set_facecolor("white")

ax=fig_forecast.gca()
# disable scientific numbers
ax.ticklabel_format(useOffset=False, style='plain')

german_population = 83020000
german_population_herd = german_population * 0.7

# first_date = df_kumulative["ds"].loc[0]
# last_date = df_kumulative["ds"].loc[len(df_kumulative) - 1]

# print(first_date, last_date, german_population)

ax.axhline(y=german_population, xmin=0, xmax=1)

# ax.plot(df_kumulative["ds"], german_population, '-g')
# ax.plot(first_date, german_population_herd, last_date, german_population_herd, marker = 'o')

prophet_plot_forecast_fig = prophet.plot(forecast, ax=ax);

prophet_plot_forecast_fig.savefig('output/forecast.png')

In [ ]:
prophet.plot_components(forecast);